In [1]:
import os
import numpy
from datetime import datetime, timedelta

In [2]:
ws_folder = '/disk1/workspace/20240116'
MYD_RAA_infos_npy = os.path.join(ws_folder, 'analysis', 'MYD_RAA_AU.npy')

In [3]:
MYD_matched_infos = numpy.load(MYD_RAA_infos_npy)

In [4]:
MYD_matched_infos[0]

array(['123.78', '-33.85', '201805020605', '201805020610', '44.05',
       '43.31', '256.01', '28.56', '318.29', '316.3'], dtype='<U12')

In [5]:
pixel_res = 0.01
ahi_llon = 85.0
ahi_llat = 60.0

MCD_landcover_npy = os.path.join(ws_folder, 'reference', 'MCD12Q1.006.LC.AHI01km.npy')
mcd_landcover = numpy.load(MCD_landcover_npy)

In [6]:
cloudmask_folder = '/disk4/AHI_CloudMask'

def is_clearsky_record(matched_info_item):    
    lon = float(matched_info_item[0])
    lat = float(matched_info_item[1])
    ahi_time_str = matched_info_item[3]

    ahi_cm_dat = os.path.join(cloudmask_folder, ahi_time_str[:6], 'AHIcm.v0.'+ahi_time_str+'.dat')
    cloudmask = None
    cloudmask_res = 0.02
    if not os.path.exists(ahi_cm_dat):
        print('no file:', ahi_cm_dat)
        cloudmask = numpy.zeros((6000, 6000))
    else:
        try:
            cloudmask = numpy.fromfile(ahi_cm_dat, dtype=numpy.float32)
            cloudmask[cloudmask < 0.95] = 0.   # cloud
            cloudmask[cloudmask >= 0.95] = 1.
            cloudmask = cloudmask.reshape(6000, 6000)   # 0.02°
        except Exception as e:
            print(e)
            cloudmask = numpy.zeros((6000, 6000))

    cm_lon_idx = int((lon-85.)/cloudmask_res)
    cm_lat_idx = int((60.-lat)/cloudmask_res)
#     cm_ex = 2
    cm_ex = 5
    cm_roi_ex = cloudmask[cm_lat_idx-cm_ex:cm_lat_idx+cm_ex,cm_lon_idx-cm_ex:cm_lon_idx+cm_ex]
    is_clearsky = cm_roi_ex.min()
    
    return is_clearsky, ahi_time_str

In [7]:
filter_infos = []
ahi_time_list = []
for raa_matched_info in MYD_matched_infos:
    # within AU 30S region
    lon = round(int(float(raa_matched_info[0])*100)/100,2)
    lat = round(int(float(raa_matched_info[1])*100)/100,2)
    if lon > 110 and lon < 125 and lat > -38 and lat < -28:
        # landcover
        lon_idx = int((lon - ahi_llon)/pixel_res)
        lat_idx = int((ahi_llat - lat)/pixel_res)
        landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
        if landcover_idx in [10, 2]:
            # between May to August
            month_str = raa_matched_info[2][4:6]
            if float(month_str) >= 5 and float(month_str) <= 8:                
                # is clear-sky
                is_clearsky, ahi_time_str = is_clearsky_record(raa_matched_info)
                if is_clearsky:
                    matched_info = raa_matched_info[:]
                    matched_info[0] = str(lon)
                    matched_info[1] = str(lat)
                    filter_infos.append(matched_info)
                    ahi_time_list.append(ahi_time_str)

In [8]:
len(MYD_matched_infos)

285856

In [9]:
len(filter_infos)

3846

In [10]:
numpy.save(os.path.join(ws_folder, 'process', 'MYD_RAA_AU_30S_5-8_clear.npy'), numpy.array(filter_infos))

In [11]:
ahi_ac_list = []
for filtered_item_idx in range(len(filter_infos)):
    filtered_item = filter_infos[filtered_item_idx]
    lon = float(filtered_item[0])
    lat = float(filtered_item[1])
    ahi_time = ahi_time_list[filtered_item_idx]
    # landcover
    lon_idx = int((lon - ahi_llon)/pixel_res)
    lat_idx = int((ahi_llat - lat)/pixel_res)
    landcover_idx = int(mcd_landcover[lat_idx][lon_idx])
    ahi_ac_info = [str(lon), str(lat), ahi_time, str(landcover_idx)]
    ahi_ac_list.append(ahi_ac_info)

In [12]:
numpy.save(os.path.join(ws_folder, 'process', 'MYD_RAA_AU_30S_5-8_clear_ahi_time.npy'), numpy.array(ahi_ac_list))

In [13]:
for ahi_ax in ahi_ac_list:
    print(ahi_ax)

['120.75', '-33.52', '201805160620', '10']
['120.76', '-33.52', '201805160620', '10']
['120.77', '-33.52', '201805160620', '10']
['120.71', '-33.53', '201805160620', '10']
['120.61', '-33.54', '201805160620', '10']
['120.62', '-33.54', '201805160620', '10']
['120.63', '-33.54', '201805160620', '10']
['120.52', '-33.57', '201805160620', '10']
['120.53', '-33.57', '201805160620', '10']
['120.89', '-33.59', '201805160620', '10']
['120.84', '-33.6', '201805160620', '10']
['120.85', '-33.6', '201805160620', '10']
['120.78', '-33.61', '201805160620', '10']
['120.79', '-33.61', '201805160620', '10']
['120.8', '-33.61', '201805160620', '10']
['120.72', '-33.61', '201805160620', '10']
['120.73', '-33.61', '201805160620', '10']
['120.74', '-33.61', '201805160620', '10']
['120.78', '-33.61', '201805160620', '10']
['120.66', '-33.63', '201805160620', '10']
['120.67', '-33.63', '201805160620', '10']
['120.68', '-33.63', '201805160620', '10']
['120.69', '-33.63', '201805160620', '10']
['120.7', '-33

In [17]:
# ahi_ac_list_lc2 = []
# for ahi_ac_item in ahi_ac_list:
#     if ahi_ac_item[3] == '2':
#         ahi_ac_list_lc2.append(ahi_ac_item)
# numpy.save(os.path.join(ws_folder, 'process', 'MYD_RAA_AU_30S_5-8_clear_ahi_time_lc2.npy'), numpy.array(ahi_ac_list_lc2))